In [ ]:
# 城市名称
# 机场名称
# 航司名称
# 飞机型号
# 飞机注册号

In [1]:
from utils import *
from decimal import *
import pandas as pd
from tqdm import tqdm
from Q_templates import Q_templates_easy
import json
class DecimalEncoder(json.JSONEncoder):
    def default(self,o):
        if isinstance(o,Decimal):
            return float(o)
        super(DecimalEncoder,self).default(o)

In [2]:
connection = connect_to_database("ai_database_dev")
cursor = connection.cursor()

In [3]:
connection1 = connect_to_database("data_center_dev")
cursor1 = connection1.cursor()

In [ ]:
connection2 = connect_to_database("data_center_release")
cursor2 = connection2.cursor()

In [ ]:
city_names_airport = [_.strip() for _ in open("meta_data/city_name_airport.txt").readlines()]
city_names_supplier = [_.strip() for _ in open("meta_data/city_name_supplier.txt").readlines()]
aircraft_registrations = [_.strip() for _ in open("meta_data/aircraft_registration.txt").readlines()]
aircraft_model_names = [_.strip() for _ in open("meta_data/aircraft_model_name.txt").readlines()]
supplier_names = [_.strip() for _ in open("meta_data/supplier_name.txt").readlines()]
airport_names = [_.strip() for _ in open("meta_data/airport_name.txt").readlines()]

In [ ]:
Q_template = "{city_name}有哪些机场？请给出机场名称和对应的四字码。"
SQL_template = \
'''SELECT DISTINCT airport_name, icao_code FROM airport_fbo_details WHERE city_name = "{}";'''

dataset = []

for city_name in tqdm(city_names_airport):
    Q = Q_template.replace("{city_name}", city_name)
    SQL = SQL_template.format(city_name)
    cursor.execute(SQL)
    A = pd.DataFrame(cursor.fetchall(), columns=["airport_name", "icao_code"]).values.tolist()
    if A:
        A = [f"机场：{_[0]}, 四字码：{_[1]}" for _ in A]
        dataset.append({"Q": Q, "A": A, "SQL": SQL})

In [ ]:
# 一次性查询所有数据，避免每次循环单独查询
SQL = 'SELECT DISTINCT city_name, airport_name, icao_code FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['city_name', 'airport_name', 'icao_code'])

# 创建一个包含所有问题和答案的列表
dataset = []
for city_name in tqdm(city_names_airport):
    # 过滤数据
    airports = df[df['city_name'] == city_name]['airport_name'].tolist()
    icao_codes = df[df['city_name'] == city_name]['icao_code'].tolist()
    if airports:
        Q = f"{city_name}有哪些机场？请给出机场名称和对应的四字码。"
        SQL_query = f'SELECT DISTINCT airport_name, icao_code FROM airport_fbo_details WHERE city_name = "{city_name}";'
        A = [f"机场名称：{airport}, 四字码：{icao_code}" for airport, icao_code in zip(airports, icao_codes)]
        dataset.append({"Q": Q, "A": A, "SQL": SQL_query})

import json
with open("synthetic_data/1-4.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)


In [ ]:
Q_template = "飞机注册号为{aircraft_registration}的飞机是哪一家公司的？"
SQL_template = \
'''SELECT sd.supplier_name
FROM aircraft_details ad
JOIN supplier_details sd ON ad.supplier_id = sd.supplier_id
WHERE ad.registration = "{}";'''

dataset = []

for aircraft_registration in tqdm(aircraft_registrations):
    Q = Q_template.replace("{aircraft_registration}", aircraft_registration)
    SQL = SQL_template.format(aircraft_registration)
    cursor.execute(SQL)
    A = pd.DataFrame(cursor.fetchall())[0].tolist()
    if A:
        dataset.append({"Q": Q, "A": A, "SQL": SQL})

In [ ]:
# 一次性查询所有数据，避免每次循环单独查询
SQL = '''
SELECT ad.registration, sd.supplier_name
FROM aircraft_details ad
JOIN supplier_details sd ON ad.supplier_id = sd.supplier_id;
'''
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['registration', 'supplier_name'])

# 创建一个包含所有问题和答案的列表
dataset = []
for aircraft_registration in tqdm(aircraft_registrations):
    # 过滤数据
    supplier = df[df['registration'] == aircraft_registration]['supplier_name'].tolist()
    if supplier:
        supplier = supplier[0]
        Q = f"飞机注册号为{aircraft_registration}的飞机是哪一家公司的？"
        SQL_query = f'SELECT sd.supplier_name FROM aircraft_details ad JOIN supplier_details sd ON ad.supplier_id = sd.supplier_id WHERE ad.registration = "{aircraft_registration}";'
        dataset.append({"Q": Q, "A": supplier, "SQL": SQL_query})

with open("synthetic_data/1-5.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)


In [ ]:
city_names_supplier[190]

In [ ]:
Q_template = "{city_name}的航空公司有哪些？"
SQL_template = \
'''SELECT supplier_name FROM supplier_details WHERE supplier_city_name = "{}";'''

dataset = []

for city_name in tqdm(city_names_supplier):
    Q = Q_template.replace("{city_name}", city_name)
    SQL = SQL_template.format(city_name)
    cursor.execute(SQL)
    try:
        A = pd.DataFrame(cursor.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL})
    except:
        print(city_name)
        print(A)
        print(SQL)
        continue

with open("synthetic_data/1-6.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
Q_template = "{aircraft_model_names}的最大载客量是多少？"
SQL_template = \
'''SELECT max(seat_num) FROM aircraft_details WHERE aircraft_model_name = "{}";'''

dataset = []

for aircraft_model_name in tqdm(aircraft_model_names):
    Q = Q_template.replace("{aircraft_model_names}", aircraft_model_name)
    SQL = SQL_template.format(aircraft_model_name)
    cursor.execute(SQL)
    try:
        A = pd.DataFrame(cursor.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL})
    except:
        print(aircraft_model_name)
        print(A)
        print(SQL)
        continue

with open("synthetic_data/1-7.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
Q_template = "{supplier_name}航司的总部位于哪个城市？"
SQL_template = \
'''SELECT supplier_city_name FROM supplier_details WHERE supplier_name = "{}";'''

dataset = []

for supplier_name in tqdm(supplier_names):
    Q = Q_template.replace("{supplier_name}", supplier_name)
    SQL = SQL_template.format(supplier_name)
    cursor.execute(SQL)
    try:
        A = pd.DataFrame(cursor.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL})
    except:
        print(supplier_name)
        print(A)
        print(SQL)
        continue

with open("synthetic_data/1-8.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
# 一次性查询所有供应商及其所在城市的数据
SQL = 'SELECT supplier_name, supplier_city_name FROM supplier_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['supplier_name', 'supplier_city_name'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for supplier_name in tqdm(supplier_names):
    try:
        # 在 DataFrame 中过滤出对应的城市
        A = df[df['supplier_name'] == supplier_name]['supplier_city_name'].tolist()[0]
        if A:
            Q = f"{supplier_name}航司的总部位于哪个城市？"
            SQL_query = f'SELECT supplier_city_name FROM supplier_details WHERE supplier_name = "{supplier_name}";'
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except Exception as e:
        print(f"Error processing supplier: {supplier_name}, Error: {e}")
        continue

# 将结果保存到文件
output_path = "synthetic_data/1-8.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和SQL查询的模板
Q_template = "{airport_name}机场的三字码是什么？"
SQL_template = '''SELECT iata_code FROM airport_fbo_details WHERE airport_name = "{}";'''

# 一次性查询所有机场的名称和对应的三字码
SQL = 'SELECT airport_name, iata_code FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['airport_name', 'iata_code'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for airport_name in tqdm(airport_names):
    try:
    # 过滤出对应机场的三字
        A = df[df['airport_name'] == airport_name]['iata_code'].tolist()[0]
        if A:
            Q = Q_template.format(airport_name=airport_name)
            SQL_query = SQL_template.format(airport_name)
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except Exception as e:
        print(f"Error processing airport: {airport_name}, Error: {e}")
        continue

# 保存到文件
with open('synthetic_data/1-9.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
Q_template = "{aircraft_registration}的制造商是谁？"
SQL_template = '''SELECT brand_name FROM aircraft_details WHERE registration = "{}";'''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for aircraft_registration in tqdm(aircraft_registrations):
    Q = Q_template.format(aircraft_registration=aircraft_registration)
    SQL_query = SQL_template.format(aircraft_registration)
    cursor.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except:
        print(A)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-14.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)


In [ ]:
Q_template = "{aircraft_model_name}的翼展是多少？"
SQL_template = '''SELECT aircraft_width FROM dc_aircraft_model WHERE name = "{}" AND aircraft_width IS NOT NULL;'''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    Q = Q_template.format(aircraft_model_name=aircraft_model_name)
    SQL_query = SQL_template.format(aircraft_model_name)
    cursor1.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor1.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except:
        print(A)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-20.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
Q_template = "{aircraft_model_name}的起飞重量是多少？"
SQL_template = '''SELECT max_takeoff_weight FROM dc_aircraft_model WHERE name = "{}" AND max_takeoff_weight IS NOT NULL;  '''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    Q = Q_template.format(aircraft_model_name=aircraft_model_name)
    SQL_query = SQL_template.format(aircraft_model_name)
    cursor1.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor1.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except Exception as e:
        print(A)
        print(e)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-21.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
Q_template = "{aircraft_model_name}属于哪种机型类型（如远程、小飞机）？"
SQL_template = '''SELECT aircraft_type_match.aircraft_type FROM aircraft_type_match INNER JOIN dc_aircraft_model ON aircraft_type_match.`code` = dc_aircraft_model.aircraft_type WHERE name = "{}" LIMIT 1;
'''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    Q = Q_template.format(aircraft_model_name=aircraft_model_name)
    SQL_query = SQL_template.format(aircraft_model_name)
    cursor2.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor2.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except:
        print(A)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-23.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
Q_template = "{aircraft_model_name}的飞行高度范围是多少？"
SQL_template = '''SELECT certified_ceiling FROM dc_aircraft_model WHERE name = "{}" AND certified_ceiling IS NOT NULL LIMIT 1;'''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    Q = Q_template.format(aircraft_model_name=aircraft_model_name)
    SQL_query = SQL_template.format(aircraft_model_name)
    cursor2.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor2.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except:
        print(A)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-24.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
Q_template = "Fleet information of {supplier_name}？"
SQL_template = '''SELECT aircraft_details.aircraft_model_name FROM aircraft_details JOIN supplier_details ON aircraft_details.supplier_id = supplier_details.supplier_id WHERE supplier_details.supplier_name = "{}";'''

# 初始化数据集
dataset = []
# 构建问题和答案的字典
for supplier_name in tqdm(supplier_names):
    Q = Q_template.format(supplier_name=supplier_name)
    SQL_query = SQL_template.format(supplier_name)
    cursor.execute(SQL_query)
    try:
        A = pd.DataFrame(cursor.fetchall())[0].tolist()
        if A:
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except:
        print(A)
        # print(SQL)
        continue

# 保存到文件
with open('synthetic_data/1-25.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
# 问题和SQL查询的模板
Q_template = "{airport_name}机场的四字码是什么？"
SQL_template = '''SELECT icao_code FROM airport_fbo_details WHERE airport_name = "{}";'''

# 一次性查询所有机场的名称和对应的四字码
SQL = 'SELECT airport_name, icao_code FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['airport_name', 'icao_code'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for airport_name in tqdm(airport_names):
    try:
        A = df[df['airport_name'] == airport_name]['icao_code'].tolist()[0]
        if A:
            Q = Q_template.format(airport_name=airport_name)
            SQL_query = SQL_template.format(airport_name)
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except Exception as e:
        print(f"Error processing airport: {airport_name}, Error: {e}")
        continue

# 保存到文件
with open('synthetic_data/1-10.json', "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和SQL查询的模板
Q_template = "{airport_name}机场在哪个国家哪个城市？"
SQL_template = '''SELECT country_name, city_name FROM airport_fbo_details WHERE airport_name = "{}";'''

# 一次性查询所有机场的名称、国家和城市
SQL = 'SELECT airport_name, country_name, city_name FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['airport_name', 'country_name', 'city_name'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for airport_name in tqdm(airport_names):
    try:
        # 查找该机场的国家和城市信息
        result = df[df['airport_name'] == airport_name][['country_name', 'city_name']].values
        if result.size > 0:
            country_name, city_name = result[0]
            Q = Q_template.format(airport_name=airport_name)
            SQL_query = SQL_template.format(airport_name)
            A = f"{city_name}, {country_name}"
            dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
    except Exception as e:
        print(f"Error processing airport: {airport_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-12.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{airport_name}机场的跑道数量是多少？"
SQL_template = '''SELECT runway_num FROM airport_fbo_details WHERE airport_name = "{}";'''

# 一次性查询所有机场名称和对应的跑道数量
SQL = 'SELECT airport_name, runway_num FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['airport_name', 'runway_num'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for airport_name in tqdm(airport_names):
    try:
        # 查找该机场的跑道数量
        result = df[df['airport_name'] == airport_name]['runway_num'].tolist()
        if result:
            runway_num = result[0]
            Q = Q_template.format(airport_name=airport_name)
            SQL_query = SQL_template.format(airport_name)
            A = int(runway_num)
            if A:
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Airport: {airport_name} has no runway number available")
    except Exception as e:
        print(f"Error processing airport: {airport_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-11.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")

In [ ]:
SQL = 'SELECT DISTINCT city_name, airport_name, icao_code FROM airport_fbo_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['city_name', 'airport_name', 'icao_code'])
Q_template = "停场在{airport_name}的所有航司有哪些？"
SQL_template = \
'''SELECT
    DISTINCT supplier_name
FROM
    zts_plane_status
WHERE
    status = 0
    AND supplier_name IS NOT NULL
    AND pure_reg IN (
        SELECT
            LOWER(REPLACE(REPLACE(registration, ' ', ''), '-', ''))
        FROM
            dc_aircraft
        WHERE
            delete_tag = 0
    )
    AND arr_icao = "{}" COLLATE utf8mb4_general_ci;'''

dataset = []

for airport_name in tqdm(airport_names):
    try:
        icao_l = df[df['airport_name'] == airport_name]['icao_code'].tolist()
        if len(icao_l)>0:
            icao_code = icao_l[0]
        else:
            continue
        Q = Q_template.replace("{airport_name}", airport_name)
        SQL = SQL_template.format(icao_code)
        cursor1.execute(SQL)
        if cursor1.fetchone() is not None:
            A = pd.DataFrame(cursor1.fetchall())[0].tolist()
            dataset.append({"Q": Q, "A": A, "SQL": SQL})
            print(len(dataset))
    except:
        # print(airport_name)
        # print(A)
        # print(SQL)
        continue

with open("synthetic_data/1-13.json", "w") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{supplier_name}航司的官网链接是什么？"
SQL_template = '''SELECT supplier_website FROM supplier_details WHERE supplier_name = "{}";'''

# 一次性查询所有供应商的名称和对应的官网链接
SQL = 'SELECT supplier_name, supplier_website FROM supplier_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['supplier_name', 'supplier_website'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for supplier_name in tqdm(supplier_names):
    try:
        # 查找该供应商的官网链接
        result = df[df['supplier_name'] == supplier_name]['supplier_website'].tolist()
        if result:
            website = result[0]
            Q = Q_template.format(supplier_name=supplier_name)
            SQL_query = SQL_template.format(supplier_name)
            A = website
            if A:
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Supplier: {supplier_name} has no website")
    except Exception as e:
        print(f"Error processing supplier: {supplier_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-16.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{aircraft_model_name}机型的最大飞行距离是多少？"
SQL_template = '''SELECT max_range FROM aircraft_details WHERE aircraft_model_name = "{}";'''

# 一次性查询所有飞机型号的名称和对应的最大飞行距离
SQL = 'SELECT aircraft_model_name, max_range FROM aircraft_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['aircraft_model_name', 'max_range'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    try:
        # 查找该飞机型号的最大飞行距离
        result = df[df['aircraft_model_name'] == aircraft_model_name]['max_range'].tolist()
        if result:
            max_range = result[0]
            Q = Q_template.format(aircraft_model_name=aircraft_model_name)
            SQL_query = SQL_template.format(aircraft_model_name)
            if max_range:
                A = f"{int(max_range)}公里"
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Aircraft model: {aircraft_model_name} has no max range")
    except Exception as e:
        print(f"Error processing aircraft model: {aircraft_model_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-17.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{aircraft_model_name}机型的座位数是多少？"
SQL_template = '''SELECT seat_num FROM aircraft_details WHERE aircraft_model_name = "{}";'''

# 一次性查询所有飞机型号的名称和对应的座位数
SQL = 'SELECT aircraft_model_name, seat_num FROM aircraft_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['aircraft_model_name', 'seat_num'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    try:
        # 查找该飞机型号的座位数
        result = df[df['aircraft_model_name'] == aircraft_model_name]['seat_num'].tolist()
        if result:
            seat_num = result[0]
            Q = Q_template.format(aircraft_model_name=aircraft_model_name)
            SQL_query = SQL_template.format(aircraft_model_name)
            A = seat_num
            if A:
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Aircraft model: {aircraft_model_name} has no seat number")
    except Exception as e:
        print(f"Error processing aircraft model: {aircraft_model_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-18.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{aircraft_model_name}机型的机身长度是多少？"
SQL_template = '''SELECT aircraft_length FROM aircraft_details WHERE aircraft_model_name = "{}";'''

# 一次性查询所有飞机型号的名称和对应的机身长度
SQL = 'SELECT aircraft_model_name, aircraft_length FROM aircraft_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['aircraft_model_name', 'aircraft_length'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    try:
        # 查找该飞机型号的机身长度
        result = df[df['aircraft_model_name'] == aircraft_model_name]['aircraft_length'].tolist()
        if result:
            aircraft_length = result[0]
            Q = Q_template.format(aircraft_model_name=aircraft_model_name)
            SQL_query = SQL_template.format(aircraft_model_name)
            if aircraft_length:
                A = f"{aircraft_length} meters"  # 假设机身长度的单位为“meters”
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Aircraft model: {aircraft_model_name} has no aircraft length")
    except Exception as e:
        print(f"Error processing aircraft model: {aircraft_model_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-19.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "注册号为{aircraft_registration}的飞机的制造年份是什么？"
SQL_template = '''SELECT make_year FROM aircraft_details WHERE registration = "{}";'''

# 一次性查询所有飞机注册号和对应的制造年份
SQL = 'SELECT registration, make_year FROM aircraft_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['registration', 'make_year'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for aircraft_registration in tqdm(aircraft_registrations):
    try:
        # 查找该飞机注册号的制造年份
        result = df[df['registration'] == aircraft_registration]['make_year'].tolist()
        if result:
            make_year = result[0]
            Q = Q_template.format(aircraft_registration=aircraft_registration)
            SQL_query = SQL_template.format(aircraft_registration)
            if make_year:
                A = f"{int(make_year)}年"
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Aircraft registration: {aircraft_registration} has no make year")
    except Exception as e:
        print(f"Error processing registration: {aircraft_registration}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-22.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 问题和 SQL 查询的模板
Q_template = "{aircraft_model_name}机型属于哪种机型类型（如远程、小飞机）？"
SQL_template = '''SELECT aircraft_type FROM aircraft_details WHERE aircraft_model_name = "{}";'''

# 一次性查询所有飞机型号的名称和对应的机型类型
SQL = 'SELECT aircraft_model_name, aircraft_type FROM aircraft_details;'
cursor.execute(SQL)
df = pd.DataFrame(cursor.fetchall(), columns=['aircraft_model_name', 'aircraft_type'])

# 初始化数据集
dataset = []

# 构建问题和答案的字典
for aircraft_model_name in tqdm(aircraft_model_names):
    try:
        # 查找该飞机型号的机型类型
        result = df[df['aircraft_model_name'] == aircraft_model_name]['aircraft_type'].tolist()
        if result:
            aircraft_type = result[0]
            Q = Q_template.format(aircraft_model_name=aircraft_model_name)
            SQL_query = SQL_template.format(aircraft_model_name)
            A = aircraft_type
            if A:
                dataset.append({"Q": Q, "A": A, "SQL": SQL_query})
            else:
                print(f"Aircraft model: {aircraft_model_name} has no aircraft type")
    except Exception as e:
        print(f"Error processing aircraft model: {aircraft_model_name}, Error: {e}")
        continue

# 保存到文件
output_path = 'synthetic_data/1-16.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

print(f"Data successfully saved to {output_path}")


In [ ]:
# 2-1
Q_template = "{supplier_name}在{year}年每个月的飞行量是多少？按从大到小排列。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''SELECT
    p.date_month,
    COUNT(*) AS num
FROM
    zts_plane_flight_1 AS p
INNER JOIN
    aircraft_details AS a
ON
    p.pure_reg = a.pure_reg
WHERE
    a.supplier_id = "{}" AND p.date_month BETWEEN '{}-01' AND '{}-12'
GROUP BY
    p.date_month
ORDER BY
    num DESC;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[6:25]):
    for year in years:
        SQL = SQL_template.format(supplier['supplier_id'],year,year)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        # print(Q)
        # print(SQL)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-1.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
# 2-2 待测试（超过90s限制）
Q_template = "{supplier_name}最繁忙的前10架飞机在{year}年每个月的飞行量分别是多少？按注册号排序。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH Top10_PureRegs AS (
    SELECT
        p.pure_reg,
        COUNT(*) AS num
    FROM
        zts_plane_flight_1 AS p
    INNER JOIN
        aircraft_details AS a
    ON
        p.pure_reg = a.pure_reg
    WHERE
        a.supplier_id = {}
    GROUP BY
        p.pure_reg
    ORDER BY
        num DESC
    LIMIT 10
)
SELECT
    p.pure_reg,
    p.date_month,
    COUNT(*) AS num
FROM
    zts_plane_flight_1 AS p
INNER JOIN
    Top10_PureRegs AS t
ON
    p.pure_reg = t.pure_reg
WHERE
    p.date_month BETWEEN '{}-01' AND '{}-12'
GROUP BY
    p.pure_reg,
    p.date_month
ORDER BY
    p.pure_reg ASC,
    p.date_month ASC;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        SQL = SQL_template.format(supplier['supplier_id'],year,year)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        # print(Q)
        print(SQL)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-2.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
# 2-3 放弃
Q_template = "{supplier_name}的飞行量在{year}的{month1}月到{month2}月之间有何变化？变化率是多少？"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH Top10_PureRegs AS (
    SELECT
        p.pure_reg,
        COUNT(*) AS num
    FROM
        zts_plane_flight_1 AS p
    INNER JOIN
        aircraft_details AS a
    ON
        p.pure_reg = a.pure_reg
    WHERE
        a.supplier_id = {}
    GROUP BY
        p.pure_reg
    ORDER BY
        num DESC
    LIMIT 10
)
SELECT
    p.pure_reg,
    p.date_month,
    COUNT(*) AS num
FROM
    zts_plane_flight_1 AS p
INNER JOIN
    Top10_PureRegs AS t
ON
    p.pure_reg = t.pure_reg
WHERE
    p.date_month BETWEEN '{}-01' AND '{}-12'
GROUP BY
    p.pure_reg,
    p.date_month
ORDER BY
    p.pure_reg ASC,
    p.date_month ASC;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        SQL = SQL_template.format(supplier['supplier_id'],year,year)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        # print(Q)
        # print(SQL)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
output_path = 'synthetic_data/2-2.json'
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
# 2-6
Q_template = "请提供注册号为{registration}的飞机在{year}年每个月的飞行量是多少？按月份排序。"
DB = 'ai_database_dev'
pureReg_csv = "meta_data/pure_reg2id.csv"
df = pd.read_csv(pureReg_csv)
dataset = []
pureReg_dict_list = df[['aircraft_id', 'pure_reg']].to_dict('records')
SQL_template = '''SELECT
        date_month,
        count(*) AS num
FROM
        zts_plane_flight_1
WHERE
        pure_reg COLLATE utf8mb4_0900_ai_ci= "{}" COLLATE utf8mb4_0900_ai_ci
        AND date_month BETWEEN '{}-01'
        AND '{}-12'
GROUP BY
        date_month
ORDER BY
        date_month ASC;'''
years = [2024]
for pureReg in tqdm(pureReg_dict_list[:25]):
    for year in years:
        SQL = SQL_template.format(pureReg['pure_reg'],year,year)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{registration}", pureReg['pure_reg'])
        # print(Q)
        print(SQL)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-6.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
# 2-7
Q_template = "请提供注册号为{registration}的飞机在{year}年{month1}月至{month2}月的飞行量是多少？按月份排序。"
DB = 'ai_database_dev'
pureReg_csv = "meta_data/pure_reg2id.csv"
df = pd.read_csv(pureReg_csv)
dataset = []
pureReg_dict_list = df[['aircraft_id', 'pure_reg']].to_dict('records')
SQL_template = '''SELECT
    pure_reg,
    ROUND(SUM(minutes_diff) / 60, 2) AS fly_hours
FROM
    zts_plane_flight_1
WHERE
    pure_reg = "{}" COLLATE utf8mb4_0900_ai_ci
    AND arr_date BETWEEN "{}" AND "{}" COLLATE utf8mb4_0900_ai_ci
GROUP BY
    pure_reg;'''
years = [2024]
month1 = '-01-01'
month2 = '-03-01'
for pureReg in tqdm(pureReg_dict_list[:25]):
    for year in years:
        SQL = SQL_template.format(pureReg['pure_reg'], str(year) + month1, str(year) + month2)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{month1}", str(1))
        Q = Q.replace("{month2}", str(3))
        Q = Q.replace("{registration}", pureReg['pure_reg'])
        # print(Q)
        print(SQL)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-6.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4)

In [ ]:
# 2-8
Q_template = "{supplier_name}在{year}年每个月各机型的平均飞行小时数分别是多少？"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''SELECT
        a.date_month,
        b.aircraft_model_name,
        round(( sum( a.diff_minutes )/ count(*))/ 60, 2 ) AS average_hours
FROM
        zts_plane_flight_1 a
        JOIN aircraft_details b ON a.pure_reg = b.pure_reg
WHERE
        b.supplier_id = "{}"
        AND date_month BETWEEN '{}-01'
        AND '{}-12'
        AND a.diff_minutes BETWEEN 30
        AND 1080
GROUP BY
        a.date_month,
        b.aircraft_model_name
ORDER BY
        date_month ASC,
        aircraft_model_name DESC;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        SQL = SQL_template.format(supplier['supplier_id'],year,year)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        # print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-8.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-10
Q_template = "列出注册号为{registration}的飞机在{year}年所有航班的详细信息。"
DB = 'ai_database_dev'
pureReg_csv = "meta_data/pure_reg2id.csv"
df = pd.read_csv(pureReg_csv)
dataset = []
pureReg_dict_list = df[['aircraft_id', 'pure_reg']].to_dict('records')
SQL_template = '''SELECT
        *
FROM
        zts_plane_flight_1
WHERE
        pure_reg ="{}" COLLATE utf8mb4_0900_ai_ci
        AND date_month COLLATE utf8mb4_0900_ai_ci BETWEEN '{}-01'
        AND '{}-12'
        AND diff_minutes COLLATE utf8mb4_0900_ai_ci BETWEEN 30
        AND 1080;'''
years = [2024]
for pureReg in tqdm(pureReg_dict_list[3:28]):
    for year in years:
        SQL = SQL_template.format(pureReg['pure_reg'],year,year)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{registration}", pureReg['pure_reg'])
        # print(Q)
        print(SQL)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except Exception as e:
            print(e)
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-10.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-11
Q_template = "{supplier_name}在{year}年运营的航线中，前五条最繁忙的航线及其飞行次数是多少？"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''SELECT
        a.dep_icao,
        a.arr_icao,
        count(*) AS num
FROM
        zts_plane_flight_1 a
        JOIN aircraft_details b ON a.pure_reg = b.pure_reg
WHERE
        b.supplier_id = "{}"
        AND a.date_month COLLATE utf8mb4_0900_ai_ci BETWEEN '{}-01'
        AND '{}-12'
GROUP BY
        a.dep_icao,
        a.arr_icao
ORDER BY
        num DESC
        LIMIT 5;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        SQL = SQL_template.format(supplier['supplier_id'],year,year)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        # print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-11.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-12
Q_template = "{supplier_name}在{year}年每个月飞往{country_name}的航班数量是怎样的？请按环比变化排序。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
country_csv = "meta_data/country_id.csv"
df = pd.read_csv(supplier_csv)
df1 = pd.read_csv(country_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
country_dict_list = df1[['country_name', 'country_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                b.supplier_id,
                a.arr_country_id,
                COUNT(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN aircraft_details b ON a.pure_reg = b.pure_reg
        WHERE
                b.supplier_id = "{}"
                AND a.arr_country_id = "{}"
                AND a.date_month BETWEEN '{}-01'
                AND '{}-12'
        GROUP BY
                a.date_month,
                b.supplier_id,
                a.arr_country_id
        ),
        biao_2 AS ( SELECT *, LAG( num ) OVER ( PARTITION BY supplier_id, arr_country_id ORDER BY date_month ) AS last_num FROM biao_1 ),
        biao_3 AS (
        SELECT
                date_month,
                supplier_id,
                arr_country_id,
                num,
                last_num,
                ROUND((( num - last_num ) / last_num ) * 100, 2 ) AS rate_value
        FROM
                biao_2
        WHERE
                last_num > 0
        ) SELECT
        date_month,
        supplier_id,
        arr_country_id,
        num,
        last_num,
        CONCAT( rate_value, '%' ) AS rate
FROM
        biao_3
ORDER BY
        rate_value DESC;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:25]):
    for country in country_dict_list:
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        if country['country_name'] != 'United States':
            continue
        # @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
        for year in years:
            SQL = SQL_template.format(supplier['supplier_id'],country['country_id'],year,year)
            Q = Q_template.replace("{year}", str(year))
            Q = Q.replace("{supplier_name}", supplier['supplier_name'])
            Q = Q.replace("{country_name}", country['country_name'])
            # print(Q)
            A = None
            try:
                cursor.execute(SQL)
                A = cursor.fetchall()
            except:
                continue
            print(A)
            res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
            dataset.append(res_data)
            output_path = 'synthetic_data/2-12.json'
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-13
Q_template = "在{year}年{supplier_name}的前五大目的地国家（按航班量排序）有哪些？"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''SELECT
        arr_country_id,
        arr_country_name,
        count(*) AS num
FROM
        zts_plane_flight_1 a
        JOIN aircraft_details b ON a.pure_reg = b.pure_reg
WHERE
        a.date_month BETWEEN '{}-01'
        AND '{}-12'
        AND b.supplier_id = "{}"
GROUP BY
        arr_country_id,
        arr_country_name
ORDER BY
        num DESC
        LIMIT 5;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        SQL = SQL_template.format(year,year, supplier['supplier_id'])
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        # print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-13.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-14
Q_template = "请给出{supplier_name}在{year}年执飞{dep_name}和{arr_name}之间的所有航班详细信息，包括日期和使用的飞机型号。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
airports_csv = "meta_data/airport_name2icao.csv"
df = pd.read_csv(supplier_csv)
df1 = pd.read_csv(airports_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
airports_dict_list = df1[['airport_name', 'icao_code']].to_dict('records')
SQL_template = '''SELECT
        a.*,b.aircraft_model_name
FROM
        zts_plane_flight_1 a
        JOIN aircraft_details b ON a.pure_reg = b.pure_reg
WHERE
        b.supplier_id = "{}" COLLATE utf8mb4_0900_ai_ci
        AND a.date_month COLLATE utf8mb4_0900_ai_ci BETWEEN '{}-01'
        AND '{}-12'
        AND a.dep_icao = "{}" COLLATE utf8mb4_0900_ai_ci
        AND a.arr_icao = "{}" COLLATE utf8mb4_0900_ai_ci;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        for arr in airports_dict_list:
            for dep in airports_dict_list:
                arr_icao = arr['icao_code']
                dep_icao = dep['icao_code']
                if arr_icao != dep_icao:
                    SQL = SQL_template.format(supplier['supplier_id'], year,year, dep['icao_code'], arr['icao_code'])
                    Q = Q_template.replace("{year}", str(year))
                    Q = Q.replace("{dep_name}", dep['airport_name'])
                    Q = Q.replace("{arr_name}", arr['airport_name'])
                    # print(Q)
                    A = None
                    try:
                        cursor.execute(SQL)
                        A = cursor.fetchall()
                    except:
                        continue
                    print(A)
                    res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
                    dataset.append(res_data)
                    output_path = 'synthetic_data/2-14.json'
                    with open(output_path, "w", encoding="utf-8") as f:
                        json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)


In [ ]:
# 2-16
Q_template = "{supplier_name}在{year}年哪个月份的飞行量最大？该月的总航班数量是多少？"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''SELECT
        a.date_month,
        count(*) as num
FROM
        ai_database_dev.zts_plane_flight_1 a
        JOIN aircraft_details b ON a.pure_reg = b.pure_reg
WHERE
        b.supplier_id = "{}"
        AND a.date_month BETWEEN '{}-01'
        AND '{}-12'
GROUP BY
        a.date_month
ORDER BY
        num DESC
        LIMIT 1;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        SQL = SQL_template.format(supplier['supplier_id'],year,year)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        # print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-16.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-17
Q_template = "{supplier_name}在{year}年每个月的飞行量及其环比增长或下降百分比。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                b.supplier_id,
                COUNT(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN aircraft_details b ON a.pure_reg = b.pure_reg
        WHERE
                b.supplier_id = "{}"
                AND a.date_month BETWEEN '{}-01'
                AND '{}-12'
        GROUP BY
                a.date_month,
                b.supplier_id
        ),
        biao_2 AS ( SELECT *, LAG( num ) OVER ( PARTITION BY supplier_id ORDER BY date_month ) AS last_num FROM biao_1 ) SELECT
        date_month,
        supplier_id,
        num,
        last_num,
        concat( ROUND((( num - last_num ) / last_num ) * 100, 2 ), '%' ) AS rate
FROM
        biao_2
ORDER BY
        date_month ASC;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        SQL = SQL_template.format(supplier['supplier_id'],year,year)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        # print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-17.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-18
Q_template = "{supplier_name}在过去的{year}年飞行量变化趋势如何？是否有显著的增减？"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                b.supplier_id,
                a.date_month,
                count(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN aircraft_details b ON a.pure_reg = b.pure_reg
        WHERE
                b.supplier_id = "{}"
                AND a.date_month BETWEEN '{}-01'
                AND '{}-12'
        GROUP BY
                b.supplier_id,
                a.date_month
        ORDER BY
                a.date_month
        ),
        biao_2 AS ( SELECT *, lag( num ) over ( PARTITION BY supplier_id ORDER BY date_month ) AS last_num FROM biao_1 ) SELECT
        *,
        num - last_num AS diff_num,
        concat( ROUND((( num - last_num ) / last_num ) * 100, 2 ), '%' ) AS rate
FROM
        biao_2
ORDER BY
        date_month ASC;'''
years = [2024]
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        SQL = SQL_template.format(supplier['supplier_id'],year,year)
        Q = Q_template.replace("{year}", str(year))
        Q = Q.replace("{supplier_name}", supplier['supplier_name'])
        # print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-18.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-19
Q_template = "列出{year}年各月飞行量增幅最大的地区，并提供环比增幅。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
    SELECT
        date_month,
        arr_area_name,
        COUNT(*) AS num
    FROM
        zts_plane_flight_1
    WHERE
        date_month BETWEEN '{}-01' AND '{}-12'
        AND arr_area_name IS NOT NULL
        AND arr_area_name <> ''
    GROUP BY
        date_month,
        arr_area_name
),
biao_2 AS (
    SELECT
        date_month,
        arr_area_name,
        num,
        LAG(num) OVER (PARTITION BY arr_area_name ORDER BY date_month) AS last_num
    FROM
        biao_1
),
biao_3 AS (
    SELECT
        date_month,
        arr_area_name,
        num,
        last_num,
        CONCAT(ROUND((num - last_num) / last_num * 100, 2), '%') AS rate
    FROM
        biao_2
    WHERE
        last_num IS NOT NULL
)
SELECT
    date_month,
    arr_area_name,
    num,
    last_num,
    rate
FROM (
    SELECT
        date_month,
        arr_area_name,
        num,
        last_num,
        rate,
        ROW_NUMBER() OVER (PARTITION BY date_month ORDER BY (num - last_num) / last_num DESC) AS rn
    FROM
        biao_3
) g
WHERE
    g.rn = 1
ORDER BY
    date_month ASC;'''
years = [2023,2024]
for year in years:
        SQL = SQL_template.format(year,year)
        Q = Q_template.replace("{year}", str(year))
        # print(Q)
        A = None
        try:
            cursor.execute(SQL)
            A = cursor.fetchall()
        except:
            continue
        print(A)
        res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        dataset.append(res_data)
        output_path = 'synthetic_data/2-19.json'
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-20
Q_template = "比较{supplier_name}在{year}年在第{season1}季度与第{season2}季度的总飞行量及变化百分比。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                b.supplier_id,
                count(*) AS last_season_num
        FROM
                zts_plane_flight_1 a
                JOIN aircraft_details b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-{}'
                AND '{}-{}'
                AND b.supplier_id = '{}'
        GROUP BY
                b.supplier_id
        ),
        biao_2 AS (
        SELECT
                b.supplier_id,
                count(*) AS season_num
        FROM
                zts_plane_flight_1 a
                JOIN aircraft_details b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-{}'
                AND '{}-{}'
                AND b.supplier_id = '{}'
        GROUP BY
                b.supplier_id
        ) SELECT
        a.supplier_id,
        b.season_num,
        a.last_season_num,
        concat( round(( b.season_num - a.last_season_num ) / a.last_season_num * 100, 2 ), '%' ) AS rate
FROM
        biao_1 a
        JOIN biao_2 b ON a.supplier_id = b.supplier_id;'''
years = [2024]
jidu = [[1,3], [4,6], [7,9],[10,12]]
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        for i in range(3):
            SQL = SQL_template.format(year,jidu[i][0], year,jidu[i][1],2135,year,jidu[i+1][0], year,jidu[i+1][1],2135,)
            Q = Q_template.replace("{year}", str(year))
            Q = Q.replace("{supplier_name}", supplier['supplier_name'])
            Q = Q.replace("{season1}", str(i+1))
            Q = Q.replace("{season2}", str(i+2))
            # print(Q)
            A = None
            try:
                cursor.execute(SQL)
                A = cursor.fetchall()
            except:
                continue
            print(A)
            if A:
                res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
                dataset.append(res_data)
                output_path = 'synthetic_data/2-20.json'
                with open(output_path, "w", encoding="utf-8") as f:
                    json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-21
Q_template = "列出{year}年每月飞行次数最多的前10个机场，以及每个机场的起降航班数量。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                a.dep_icao,
                count(*) AS dep_num
        FROM
                zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-01'
                AND '{}-12'
                AND a.dep_icao IS NOT NULL
                AND a.dep_icao <> ''
        GROUP BY
                a.date_month,
                a.dep_icao
        ),
        biao_2 AS (
        SELECT
                a.date_month,
                a.arr_icao,
                count(*) AS arr_num
        FROM
                zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-01'
                AND '{}-12'
                AND a.arr_icao IS NOT NULL
                AND a.arr_icao <> ''
        GROUP BY
                a.date_month,
                a.arr_icao
        ),
        biao_3 AS (
        SELECT
                a.date_month,
                a.dep_icao AS icao_code,
                a.dep_num + b.arr_num AS total_num
        FROM
                biao_1 a
                JOIN biao_2 b ON a.dep_icao = b.arr_icao
                AND a.date_month = b.date_month
        ) SELECT
        *
FROM
        ( SELECT date_month, icao_code, total_num, ROW_NUMBER() over ( PARTITION BY date_month ORDER BY total_num DESC ) AS rn FROM biao_3 ) g
WHERE
        g.rn <= 10'''
years = [2022, 2023, 2024]
for year in years:
    SQL = SQL_template.format(year,year, year,year)
    Q = Q_template.replace("{year}", str(year))
    print(Q)
    A = None
    try:
        cursor.execute(SQL)
        A = cursor.fetchall()
    except Exception as e:
        print(e)
        continue
    res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
    dataset.append(res_data)
    output_path = 'synthetic_data/2-21.json'
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)
# for supplier in tqdm(supplier_dict_list[:25]):
#     for year in years:
#         SQL = SQL_template.format(supplier['supplier_id'],year,year)
#         Q = Q_template.replace("{year}", str(year))
#         Q = Q.replace("{supplier_name}", supplier['supplier_name'])
#         # print(Q)
#         A = None
#         try:
#             cursor.execute(SQL)
#             A = cursor.fetchall()
#         except:
#             continue
#         print(A)
#         res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
#         dataset.append(res_data)
#         output_path = 'synthetic_data/2-18.json'
#         with open(output_path, "w", encoding="utf-8") as f:
#             json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-25
Q_template = "{country_name}的前五个繁忙机场在{year}年的{month1}月到{month2}月的飞行量变化情况如何？"
DB = 'ai_database_dev'
country_csv = "meta_data/country_name2id.csv"
df1 = pd.read_csv(country_csv)
dataset = []
# 获取所有的supplier_id列的值
country_dict_list = df1[['country_name', 'country_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                a.dep_icao,
                count(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-{}'
                AND '{}-{}'
                AND ( a.dep_country_id COLLATE utf8mb4_general_ci = "{}" )
        GROUP BY
                a.date_month,
                a.dep_icao
        ),
        biao_2 AS ( SELECT dep_icao FROM biao_1 WHERE date_month = '{}-{}' ORDER BY num DESC LIMIT 5 ),
        biao_3 AS (
        SELECT
                *,
                lag( num ) over ( PARTITION BY dep_icao ORDER BY date_month ) AS last_num
        FROM
                biao_1
        WHERE
        dep_icao IN ( SELECT dep_icao FROM biao_2 )) SELECT
        dep_icao AS icao_code,
        num,
        last_num,
        concat( round(( num - last_num )/ last_num * 100, 2 ), '%' ) AS rate
FROM
        biao_3
WHERE
        last_num IS NOT NULL;'''
years = [2024]
for country in country_dict_list[40:200]:
    for month1 in range(1,2):
        for month2 in range(6,7):
            for year in years:
                SQL = SQL_template.format(year, month1, year, month2, str(country['country_id']),year, month2, )
                Q = Q_template.replace("{year}", str(year))
                Q = Q.replace("{month1}", str(month1))
                Q = Q.replace("{month2}", str(month2))
                Q = Q.replace("{country_name}", str(country['country_name']))
                print(Q)
                A = None
                try:
                    cursor.execute(SQL)
                    A = cursor.fetchall()
                except Exception as e:
                    print(e)
                    continue
                print(A)
                res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
                dataset.append(res_data)
                output_path = 'synthetic_data/2-25json'
                with open(output_path, "w", encoding="utf-8") as f:
                    json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)
        # if A:
        #     res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
        #     dataset.append(res_data)
        #     output_path = 'synthetic_data/2-23.json'
        #     with open(output_path, "w", encoding="utf-8") as f:
        #         json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)


In [ ]:
# 2-26
Q_template = "列出{year}年每月飞行次数最多的前10个的国家，并提供环比变化。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                a.dep_country_name,
                count(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-01'
                AND '{}-12'
                AND a.dep_country_name IS NOT NULL
        GROUP BY
                a.date_month,
                a.dep_country_name
        ),
        biao_2 AS ( SELECT *, lag( num ) over ( PARTITION BY dep_country_name ORDER BY date_month ) AS last_num FROM biao_1 ),
        biao_3 AS ( SELECT *, concat( round(( num - last_num )/ last_num * 100, 2 ), '%' ) AS rate FROM biao_2 ),
        biao_4 AS ( SELECT *, ROW_NUMBER() over ( PARTITION BY date_month ORDER BY num DESC ) AS rn FROM biao_3 ) SELECT
        date_month,
        dep_country_name,
        num,
        last_num,
        rate
FROM
        biao_4
WHERE
        rn <= 10
        AND last_num IS NOT NULL;'''
years = [2022, 2023, 2024]
for year in years:
    SQL = SQL_template.format(year,year)
    Q = Q_template.replace("{year}", str(year))
    print(Q)
    A = None
    try:
        cursor.execute(SQL)
        A = cursor.fetchall()
    except Exception as e:
        print(e)
        continue
    res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
    dataset.append(res_data)
    output_path = 'synthetic_data/2-26.json'
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-27
Q_template = "2024年{supplier_name}在{area_name}地区的每月总飞行量是多少？提供环比变化。"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                count(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN aircraft_details b ON a.pure_reg = b.pure_reg COLLATE utf8mb4_general_ci
        WHERE
                a.date_month BETWEEN '{}-01'
                AND '{}-12'
                AND b.supplier_id = "{}"
                AND a.dep_area_name = "{}" COLLATE utf8mb4_general_ci
        GROUP BY
                a.date_month
        ),
        biao_2 AS ( SELECT *, lag( num ) over ( ORDER BY date_month ) AS last_num FROM biao_1 ) SELECT
        date_month,
        num,
        concat( round(( num - last_num )/ last_num * 100, 2 ), '%' ) AS rate
FROM
        biao_2
WHERE
        last_num IS NOT NULL;'''
years = [2024]
area_name = ['Europe', 'North America', 'South Asia', 'Oceania', 'West Asia', 'Africa', 'South America', 'Southeast Asia', 'East Asia']
for supplier in tqdm(supplier_dict_list[1:]):
    for area in area_name:
        for year in years:
            SQL = SQL_template.format(year,year,supplier['supplier_id'],area)
            Q = Q_template.replace("{year}", str(year))
            Q = Q.replace("{supplier_name}", supplier['supplier_name'])
            Q = Q.replace("{area_name}", area)
            print(Q)
            A = None
            try:
                cursor.execute(SQL)
                A = cursor.fetchall()
            except Exception as e:
                print(e)
                continue
            print(A)
            res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
            dataset.append(res_data)
            output_path = 'synthetic_data/2-27.json'
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-28
Q_template = "哪些地区的飞行量在{year}年的{month1}月到{month2}月显著增长？"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                a.dep_area_name,
                count(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN avigo_nlp_dev.ai_target_jet b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-{}'
                AND '{}-{}'
                AND dep_area_name IS NOT NULL
        GROUP BY
                a.date_month,
                a.dep_area_name
        ),
        biao_2 AS ( SELECT *, lag( num ) over ( PARTITION BY dep_area_name ORDER BY date_month ) AS last_num FROM biao_1 ) SELECT
        dep_area_name,
        num,
        last_num,
        num - last_num AS diff_num
FROM
        biao_2
WHERE
        last_num IS NOT NULL
ORDER BY
        num - last_num DESC
        LIMIT 3;'''
years = [2024]
area_name = ['Europe', 'North America', 'South Asia', 'Oceania', 'West Asia', 'Africa', 'South America', 'Southeast Asia', 'East Asia']
for year in years:
    for month1 in ['01', '02','03','04']:
        for month2 in ['06', '07','08','09']:
            SQL = SQL_template.format(year, month1, year, month2)
            Q = Q_template.replace("{year}", str(year))
            Q = Q.replace("{month1}", month1)
            Q = Q.replace("{month2}", month2)
            print(Q)
            A = None
            try:
                cursor.execute(SQL)
                A = cursor.fetchall()
            except Exception as e:
                print(e)
                continue
            print(A)
            res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
            dataset.append(res_data)
            output_path = 'synthetic_data/2-28.json'
            with open(output_path, "w", encoding="utf-8") as f:
                json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
# 2-29
Q_template = "{supplier_name}的航班总量在{year}年每个月的变化趋势如何？"
DB = 'ai_database_dev'
supplier_csv = "meta_data/supplier_name2id.csv"
df = pd.read_csv(supplier_csv)
dataset = []
# 获取所有的supplier_id列的值
supplier_dict_list = df[['supplier_name', 'supplier_id']].to_dict('records')
SQL_template = '''WITH biao_1 AS (
        SELECT
                a.date_month,
                b.supplier_id,
                count(*) AS num
        FROM
                zts_plane_flight_1 a
                JOIN aircraft_details b ON a.pure_reg = b.pure_reg
        WHERE
                a.date_month BETWEEN '{}-01'
                AND '{}-12'
                AND b.supplier_id = "{}"
        GROUP BY
                a.date_month,
                b.supplier_id
        ),
        biao_2 AS ( SELECT *, lag( num ) over ( PARTITION BY supplier_id ORDER BY date_month ) AS last_num FROM biao_1 ) SELECT
        supplier_id,
        date_month,
        num,
        last_num,
        num - last_num AS diff_num,
        concat( round(( num - last_num )/ last_num * 100, 2 ), '%' ) AS rate
FROM
        biao_2
WHERE
        last_num IS NOT NULL
ORDER BY
        date_month;'''
years = [2024]
area_name = ['Europe', 'North America', 'South Asia', 'Oceania', 'West Asia', 'Africa', 'South America', 'Southeast Asia', 'East Asia']
for supplier in tqdm(supplier_dict_list[:25]):
    for year in years:
        for i in range(3):
            SQL = SQL_template.format(year,year,supplier['supplier_id'])
            Q = Q_template.replace("{year}", str(year))
            Q = Q.replace("{supplier_name}", supplier['supplier_name'])
            # print(Q)
            A = None
            try:
                cursor.execute(SQL)
                A = cursor.fetchall()
            except:
                continue
            print(A)
            if A:
                res_data = {"DB": DB, "Q": Q, "A": A, "SQL": SQL}
                dataset.append(res_data)
                output_path = 'synthetic_data/2-29.json'
                with open(output_path, "w", encoding="utf-8") as f:
                    json.dump(dataset, f, ensure_ascii=False, indent=4, cls=DecimalEncoder)

In [ ]:
dataset = {}

for Q_template in Q_templates_easy:
    Q = Q_template.replace("{aircraft_registration}", aircraft_registration)
    SQL = SQL_template.format(aircraft_registration)
    cursor.execute(SQL)
    A = pd.DataFrame(cursor.fetchall())[0].tolist()
    if A:
        dataset.append({"Q": Q, "A": A, "SQL": SQL})